<a href="https://colab.research.google.com/github/chopstickexe/deep-learning-from-scratch-2/blob/master/ch05.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 下準備

$$
\newcommand{\vect}[1]{\mathbf{#1}}
\newcommand{\mat}[1]{\mathbf{#1}}
$$

## 数式の表記

変数: 小文字イタリック $x$

定数: 大文字イタリック $X$

ベクトル: 小文字ローマン体太字 $\vect{x}$

行列: 大文字ローマン体太字 $\mat{X}$

## 公式実装のclone

In [1]:
!git clone --depth=1 https://github.com/oreilly-japan/deep-learning-from-scratch-2.git
import sys 
sys.path.append('deep-learning-from-scratch-2')

Cloning into 'deep-learning-from-scratch-2'...
remote: Enumerating objects: 73, done.
remote: Counting objects: 100% (73/73), done.
remote: Compressing objects: 100% (71/71), done.
remote: Total 73 (delta 13), reused 14 (delta 0), pack-reused 0
Unpacking objects: 100% (73/73), done.


# 5章 リカレントニューラルネットワーク（RNN）

## 5.3 RNNの実装（`RNN`, `TimeRNN`クラス）

順伝播は一つ前の時刻$t-1$の隠れ変数$\vect{h_{t-1}}$と，時刻$t$の入力$\vect{x_t}$を用いて以下のように求める．

$$
\vect{h_t} = \mathrm{tanh}(\vect{h_{t-1}} \mat{W_h} + \vect{x_t} \mat{W_x} + b)
$$

なお，$y = \mathrm{tanh}(x) = \frac{e^x - e^{-x}}{e^x + e~{-x}}$の微分は$1-y^2$（付録A参照）

In [2]:
class RNN:
  def __init__(self, Wx, Wh, b):
    self.params = [Wx, Wh, b]
    self.grads = [np.zeros_like(Wx), np.zeros_like(Wh), np.zeros_like(b)]
    self.cache = None

  def forward(self, x, h_prev):
    Wx, Wh, b = self.params
    t = np.dot(h_prev, Wh) + np.dot(x, Wx) + b
    h_next = np.tanh(t)

    self.cache = (x, h_prev, h_next)
    return h_next

  def backward(self, dh_next):
    Wx, Wh, b = self.params
    x, h_prev, h_next = self.cache
    
    dt = dh_next * (1 - h_next ** 2)  # d(tanh(x))/dx = 1 - y^2
    db = np.sum(dt, axis=0)  # 一つのベクトル値をミニバッチ分複製して順伝播したので，逆伝播のときはsumする．
    dWh = np.dot(h_prev.T, dt)
    dh_prev = np.dot(dt, Wh.T)
    dWx = np.dot(x.T, dt)
    dx = np.dot(dt, Wx.T)

    self.grads[0][...] = dWx
    self.grads[1][...] = dWh
    self.grads[2][...] = db

    return dx, dh_prev    

時刻$0, ..., T$分のRNNレイヤをまとめてTime RNNレイヤとして実装する．

In [4]:
class TimeRNN:
  def __init__(self, Wx, Wh, b, stateful=False):  
    # stateful=Trueの場合には前にforwardを計算して得られたhを記憶するようにする（長い時系列をTruncateしてミニバッチ化するため）
    self.params = [Wx, Wh, b]
    self.grads = [np.zeros_like(Wx), np.zeros_like(Wh), np.zeros_like(b)]
    self.layers = None

    self.h, self.dh = None, None
    self.stateful = stateful

  def set_state(self, h):
    self.h = h

  def reset_state(self):
    self.h = None

  def forward(self, xs):
    Wx, Wh, b = self.params
    N, T, D = xs.shape
    D, H = Wx.shape

    self.layers = []
    hs = np.empty((N, T, H), dtype='f')  # zerosと違って0で初期化しない（zerosよりも高速の場合がある）

    if not self.stateful or self.h is None:
      # 初期化
      self.h = np.zeros((N, H), dtype='f')

    for t in range(T):
      layer = RNN(*self.params)  # RNNクラスのinitにWx, Wh, bを展開して代入している（つまりこの辺の重みは全部共有）
      self.h = layer.forward(xs[:, t, :], self.h)  # self.hはtごとに更新される（stateful=Trueであれば、0-Tのloopを抜けて次にforwardが呼ばれたときも引き継がれている
      hs[:, t, :] = self.h
      self.layers.append(layer)

    return hs

  def backward(self, dhs):  # 逆伝播のときの入力はdh_tとdh_{t+1}になるが、dh_{t+1}はそのままクラス変数として引き継がれてくる
    Wx, Wh, b = self.params
    N, T, H = dhs.shape
    D, H = Wx.shape

    dxs = np.empty((N, T, D), dtype='f')  # 0で初期化しない行列の初期化
    dh = 0  # Trancated BPTTに基づいてRNNを学習しているため、t=Tのときにdh_{T+1}は伝播させない（そのため、dh=self.dhではない）
    grads = [0, 0, 0]
    for t in reversed(range(T)):
      layer = self.layers[t]
      dx, dh = layer.backward(dhs[:, t, :] + dh)  # 分岐して順伝播しているのでsumで逆伝播
      dxs[:, t, :] = dx

      for i, grad in enumerate(layer.grads):
        grads[i] += grad  # Wx, Wh, bはすべてのtに対して同じものを使って（＝分岐して）順伝播しているので、sumで逆伝播
    
    for i, grad in enumerate(grads):
      self.grads[i][...] = grad  # 0-Tの逆伝播が終わったら、勾配を更新

    self.dh = dh  # いまはメンバ変数に保持するだけで何もしないが、7章のseq2seqでこれを使う

    return dxs


## 5.4 時系列データを扱うレイヤの実装（`TimeEmbedding`, `TimeAffine`, `TimeSoftmaxWithLoss`クラス）

5.3で実装したTime RNNレイヤの前後にレイヤをくっつけて、t=0～T-1のEmbedding -> RNN（t=0～T-2の隠れベクトルを入力とする） -> Affine -> Softmaxでt=1～Tの単語の確率を出力する，言語モデル（RNNLM）を実装する．

t=0～T-1をまとめて処理するTime RNNと同様に，Time Embedding, Time Affine, Time Softmax With Lossレイヤを実装する．（`common/time_layers.py` 参照）

- `TimeEmbedding`: $\vect{x_{0}}, ..., \vect{x_{T-1}}$を`xs`として受け取り，単純に各$t$に対してforward, backwardを施す．
- `TimeAffine`: `TimeEmbedding`と同様に$\vect{x_{0}}, ..., \vect{x_{T-1}}$を`x`として受け取るが，まじめに各$t$に対して$\vect{x_t} \mat{W} + \vect{b}$を計算するのではなく，バッチ内の全$\vect{x}$を各行とする行列を作り（行: バッチ数N×T，列: データ次元数D）、$\mat{W}$と一回行列積を求めれば済むようにしている．backwardも同様．
- `TimeSoftmaxWithLoss`: `TimeAffine`と同様に，N×T行の行列を作ってまとめて処理．損失値は$t=0, ..., \mathrm{T}$およびバッチ$N$の中で平均を取る．

## 5.5 RNNLMの学習と評価（`SimpleRnnlm` クラス）

5.3, 5.4のクラスを使ってSimpleRnnlmクラスを実装する．

In [5]:
import numpy as np
from common.time_layers import *

class SimpleRnnlm:
  def __init__(self, vocab_size, wordvec_size, hidden_size):
    V, D, H = vocab_size, wordvec_size, hidden_size
    rn = np.random.randn

    # 重みの初期化
    embed_W = (rn(V, D) / 100).astype('f')  # 突然100で割っているが、前章までで0.01かけてたのと同じ
    rnn_Wx = (rn(D, H) / np.sqrt(D)).astype('f')  # 前の層のノード数がnのとき，1/\sqrt{n}の標準偏差を持つ分布で初期化する（Xavierの初期値の簡易実装．本当は次層のノード数も考慮するらしい）
    rnn_Wh = (rn(H, H) / np.sqrt(H)).astype('f')
    rnn_b = np.zeros(H).astype('f')
    affine_W = (rn(H, V) / np.sqrt(H)).astype('f')
    affine_b = np.zeros(V).astype('f')

    # レイヤの生成
    self.layers = [
      TimeEmbedding(embed_W),
      TimeRNN(rnn_Wx, rnn_Wh, rnn_b, stateful=True), # stateful=Trueなので、順伝播のときにバッチの切れ目で前のバッチで計算された前時刻の隠れ状態を引き継ぐ
      TimeAffine(affine_W, affine_b)
    ]
    self.loss_layer = TimeSoftmaxWithLoss()
    self.rnn_layer = self.layers[1]

    # すべての重みと勾配をリストにまとめる
    self.params, self.grads = [], []
    for layer in self.layers:
      self.params += layer.params
      self.grads += layer.grads

  def forward(self, xs, ts):
    for layer in self.layers:
      xs = layer.forward(xs)  # 4章までの実装と同じ
    loss = self.loss_layer.forward(xs, ts)
    return loss

  def backward(self, dout=1):
    dout = self.loss_layer.backward(dout)
    for layer in reversed(self.layers):
      dout = layer.backward(dout)
    return dout

  def reset_state(self):  # RNNのstateをリセット
    self.rnn_layer.reset_state()

Penn Treebankデータセットを用いた学習コードは以下の通り（`ch05/train_custom_loop.py`）．

一般に言語モデルの評価はperplexity $e^L$ で行われる．このとき$L$はNNの損失関数と同じ交差エントロピー誤差で，perplexityは低いほど良い．

In [8]:
import matplotlib.pyplot as plt
from common.optimizer import SGD
from dataset import ptb

batch_size = 10
wordvec_size = 100
hidden_size = 100
time_size = 5  # Truncated BPTTの単位時間
lr = 0.1
max_epoch = 100

# 学習データの読み込み
corpus, word_to_id, id_to_word = ptb.load_data('train')
corpus_size = 1000  # とりあえず1000語読み込む
corpus = corpus[:corpus_size]
vocab_size = int(max(corpus) + 1)

xs = corpus[:-1]  # 入力語全部（0～1000）
ts = corpus[1:]  # 入力語の次の語全部（1～1000）
data_size = len(xs)  # 一見無駄な変数だが、corpusがcorpus_size分無い場合，data_sizeはcorpus_sizeと違う値になる（Pythonのリストのスライスは範囲外の値を指定してもエラーにならない）
print(f'corpus size: {corpus_size}, vocabulary size: {vocab_size}')

# 学習時に使用する変数
max_iters = data_size // (batch_size * time_size)  # iterationごとに単語長time_sizeのバッチがbatch_size個処理される
time_idx = 0
total_loss = 0
loss_count = 0
ppl_list = []  # perplexityを保存

# モデルの生成
model = SimpleRnnlm(vocab_size, wordvec_size, hidden_size)
optimizer = SGD(lr)

# ミニバッチを時間ずらして平行に作るためのoffsetsを計算
jump = (corpus_size - 1) // batch_size  # バッチサイズで等分
offsets = [i * jump for i in range(batch_size)]

for epoch in range(max_epoch):
  for iter in range(max_iters):
    # ミニバッチの取得
    batch_x = np.empty((batch_size, time_size), dtype='i')  # とりあえずbatch_size x time_sizeの整数値行列を初期化
    batch_t = np.empty((batch_size, time_size), dtype='i')
    for t in range(time_size):
      for i, offset in enumerate(offsets):
        batch_x[i, t] = xs[(offset + time_idx) % data_size]  # time_idxはこれまでのiteration回数*timesize+tの値
        batch_t[i, t] = ts[(offset + time_idx) % data_size]
      time_idx += 1
    
    # ミニバッチを順伝播
    loss = model.forward(batch_x, batch_t)
    # 逆伝播して重み更新
    model.backward()
    optimizer.update(model.params, model.grads)
    total_loss += loss
    loss_count += 1

  # エポックごとにperplexity評価
  ppl = np.exp(total_loss / loss_count)  # e^LのLの値にはこのエポックの損失値の平均を使う
  print(f'| epoch {epoch+1} | perplexity {ppl:.2f}')
  ppl_list.append(float(ppl))
  total_loss, loss_count = 0, 0


Done
corpus size: 1000, vocabulary size: 418
| epoch 1 | perplexity 370.01
| epoch 2 | perplexity 256.26
| epoch 3 | perplexity 225.31
| epoch 4 | perplexity 216.99
| epoch 5 | perplexity 206.85
| epoch 6 | perplexity 202.89
| epoch 7 | perplexity 198.61
| epoch 8 | perplexity 196.60
| epoch 9 | perplexity 192.39
| epoch 10 | perplexity 193.08
| epoch 11 | perplexity 188.46
| epoch 12 | perplexity 191.67
| epoch 13 | perplexity 190.04
| epoch 14 | perplexity 190.16
| epoch 15 | perplexity 188.40
| epoch 16 | perplexity 185.42
| epoch 17 | perplexity 182.37
| epoch 18 | perplexity 179.93
| epoch 19 | perplexity 180.90
| epoch 20 | perplexity 183.02
| epoch 21 | perplexity 179.35
| epoch 22 | perplexity 176.31
| epoch 23 | perplexity 174.15
| epoch 24 | perplexity 174.25
| epoch 25 | perplexity 175.61
| epoch 26 | perplexity 171.65
| epoch 27 | perplexity 170.86
| epoch 28 | perplexity 168.73
| epoch 29 | perplexity 164.28
| epoch 30 | perplexity 158.02
| epoch 31 | perplexity 161.00
| e